In [18]:
# ────────────────────────────────────────────────────────────────────────────────
# Paste this entire block into one notebook cell (or script) and run top to bottom
# ────────────────────────────────────────────────────────────────────────────────

# 1. Install & import libraries
!pip install -q tensorflow tensorflow-datasets numpy matplotlib

import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# 2. Load & concatenate the Shakespeare corpus
print("Loading tiny_shakespeare dataset...")
ds = tfds.load('tiny_shakespeare', split='train', shuffle_files=True)
text = ''.join([ex['text'].decode('utf-8') for ex in tfds.as_numpy(ds)])
print(f"Total characters: {len(text)}")

# 3. Build vocabulary & encode text
vocab      = sorted(set(text))
vocab_size = len(vocab)
print(f"Vocab size: {vocab_size}")
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text], dtype=np.int32)

# 4. Hyperparameters & train/test split
seq_length  = 100
BATCH_SIZE  = 64
BUFFER_SIZE = 10000
EPOCHS      = 5
TEST_SPLIT  = 0.9

split_idx   = int(len(text_as_int) * TEST_SPLIT)
train_ints  = text_as_int[:split_idx]
test_ints   = text_as_int[split_idx:]

# 5. Dataset builder
def make_dataset(arr, seq_length, batch_size, buffer_size=0, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(arr)
    ds = ds.batch(seq_length + 1, drop_remainder=True)
    ds = ds.map(lambda chunk: (chunk[:-1], chunk[1:]))
    if shuffle:
        ds = ds.shuffle(buffer_size)
    return ds.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(train_ints, seq_length, BATCH_SIZE, BUFFER_SIZE, shuffle=True)
test_ds  = make_dataset(test_ints,  seq_length, BATCH_SIZE)

# 6. Build & compile the model (with a defined input so params are created)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None,), dtype=tf.int32),
    tf.keras.layers.Embedding(vocab_size, 256),
    tf.keras.layers.LSTM(1024, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()   # now shows non-zero params

# 7. Checkpoint callback (filename must end in .weights.h5 when save_weights_only=True)
checkpoint_dir  = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
ckpt_filepath  = os.path.join(checkpoint_dir, 'ckpt_{epoch}.weights.h5')
cp_callback     = tf.keras.callbacks.ModelCheckpoint(
    filepath=ckpt_filepath,
    save_weights_only=True,
    verbose=1
)

# 8. Train
history = model.fit(train_ds, epochs=EPOCHS, callbacks=[cp_callback])

# 9. Evaluate on test split
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Loading tiny_shakespeare dataset...
Total characters: 1003854
Vocab size: 65


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, None, 256)           │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, None, 1024)          │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, None, 65)            │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1924 - loss: 3.2337
Epoch 1: saving model to ./training_checkpoints/ckpt_1.weights.h5
139/139 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step - accuracy: 0.1929 - loss: 3.2302
Epoch 2/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.3873 - loss: 2.1121
Epoch 2: saving model to ./training_checkpoints/ckpt_2.weights.h5
139/139 ━━━━━━━━━━━━━━━━━━━━ 21s 71ms/step - accuracy: 0.3874 - loss: 2.1115
Epoch 3/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.4665 - loss: 1.8074
Epoch 3: saving model to ./training_checkpoints/ckpt_3.weights.h5
139/139 ━━━━━━━━━━━━━━━━━━━━ 21s 73ms/step - accuracy: 0.4666 - loss: 1.8070
Epoch 4/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5138 - loss: 1.6306
Epoch 4: saving model to ./training_checkpoints/ckpt_4.weights.h5
139/139 ━━━━━━━━━━━━━━━━━━━━ 12s 73ms/step - accuracy: 0.5138 - loss: 1.6304
Epoch 5/5
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5424 - loss: 1.522

In [20]:
# ─── Cell 10: Reload weights & generate ─────────────────────────────────────

import os
import glob

# 1. Find the latest .weights.h5 file
weight_files   = glob.glob(os.path.join(checkpoint_dir, '*.weights.h5'))
latest_weights = max(weight_files, key=os.path.getctime)
print(f"Loading weights from: {latest_weights}")

# 2. Rebuild the same model architecture
gen_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None,), dtype=tf.int32),
    tf.keras.layers.Embedding(vocab_size, 256),
    tf.keras.layers.LSTM(1024, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

# 3. Load weights
gen_model.load_weights(latest_weights)

# 4. Text generation function
def generate_text(model, start_string, num_generate=300, temperature=0.5):
    input_eval = tf.expand_dims([char2idx[s] for s in start_string], 0)
    generated = []
    for _ in range(num_generate):
        preds = model(input_eval)
        preds = preds[:, -1, :] / temperature
        pred_id = tf.random.categorical(preds, num_samples=1)[0,0].numpy()
        generated.append(idx2char[pred_id])
        input_eval = tf.expand_dims([pred_id], 0)
    return start_string + ''.join(generated)

# 5. Generate & display
print(generate_text(gen_model, start_string="ROMEO: ", num_generate=300, temperature=0.5))


Loading weights from: ./training_checkpoints/ckpt_5.weights.h5
ROMEO: IS:
TENERINoures whengrandorererong;

Then s waly tharorinor watho mis chand th ingit thileared s we angat the hous we ar the t t here the tharore the thor the there s t te hen fathoulaind hathe he whe t byorengn me wholed are there thes he the the the t mere he th har y

Thithealan hea the r the y 
